In [44]:
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
from datetime import datetime
from time import sleep
from random import randint

def get_url(position, location):
    template = "https://www.indeed.com/jobs?q={}&l={}"
    url = template.format(position, location)
    return url

def get_record(card):
    a_tag = card.h2.a
    job_title  = card.h2.a.span.text
    #job_title = a_tag.get('jobTitle')
    job_url = 'https://www.indeed.com' + str(a_tag.get('href'))
    job_company = card.find('span', 'companyName').text
    post_date = card.find('span', 'date').text
    
    
    record = (job_title, job_url, job_company, post_date)
    
    return record

def main(position, location):
    
    records = []
    url = get_url(position, location)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        cards = soup.find_all('div', 'job_seen_beacon')
     
            
        for card in cards:
            record = get_record(card)
            records.append(record)
    
        try:
            soup.find('a', {'aria-label':'Next'}).get('href')
        except AttributeError:
                break
    return records


In [45]:
job_roles = main('Analyst', 'remote')
df = pd.DataFrame(job_roles )
df.columns = ['Position', 'URL', 'Comapny', 'Date Posted']
df.to_csv('jobs1.csv') 